In [2]:
import numpy as np
import pandas as pd
from rec import *

### Collaborative filtering

Collaborative filtering considers behavior of other users behavior (rather than content properties) to measure similarity between users and items. It is "collaborative" in the sense that it uses the decision made by other users to predict your future ratings. Decision are made entirely by the behavior ("collaboration") of users. In this case, similar items are found by comparing them in user-space. For example, Netflix & Amazon Recommendations, Google Ads, Facebook Ads, Search, Friends Rec., News feed, Trending news, Rank Notifications, Rank Comments

### User-User Recommendations

Generate Fake user liking activity matrix

Thumbs up : 1

Thumps down : -1

Unrated : 0

In [3]:
user_data = gen_user_activity()
user_data

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  1,  0, ...,  0,  0,  0],
       [ 1,  0,  1, ...,  1,  0,  1],
       ...,
       [ 0,  0,  0, ..., -1,  1,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  1,  0,  0]])

In [4]:
user_data.shape #200 users, 30 lessons

(200, 30)

In [22]:
df_users = pd.DataFrame(user_data)
#df_users['user_id'] = 
df_users.insert(0, 'user_id', df_users.index)
df_users = df_users.add_prefix('lesson_')
df_users.rename(columns={'lesson_user_id': 'user_id'}, inplace=True)
df_users

,user_id,lesson_0,lesson_1,lesson_2,lesson_3,lesson_4,lesson_5,lesson_6,lesson_7,lesson_8,...,lesson_20,lesson_21,lesson_22,lesson_23,lesson_24,lesson_25,lesson_26,lesson_27,lesson_28,lesson_29
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,1
3,3,1,1,1,0,0,0,1,1,0,...,0,1,0,0,0,1,0,0,0,0
4,4,0,0,0,0,1,1,1,-1,0,...,1,0,0,0,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
196,196,0,1,0,0,-1,0,-1,1,1,...,-1,0,1,0,0,0,0,0,0,-1
197,197,0,0,0,0,0,-1,-1,1,1,...,0,0,0,0,0,1,0,-1,1,0
198,198,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Matrix Factorization using gradient descent

In [46]:
sgd = FunkSGD(df_users.iloc[:,1:].values, 10 )
n_iter = 10

for _ in range(n_iter):
    sgd.one_iter()
    
sgd.reconstruction_loss()

1509.9246406966502

In [43]:
results_df = pd.DataFrame( (sgd.P @ sgd.Q.T).round(1) )
#results_df['user_id'] = results_df.index
#results_df = results_df.add_prefix('lesson_')

In [44]:
results_df # rows are users, columns are lessons

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.2,0.1,0.1,-0.0,0.1,-0.0,-0.0,0.1,0.1,0.1,...,-0.0,0.1,0.0,-0.0,0.0,0.0,0.1,0.1,0.2,-0.1
1,0.0,0.1,0.1,0.0,0.1,0.2,0.1,0.2,-0.1,-0.1,...,-0.1,0.2,0.0,0.2,0.1,0.1,-0.1,-0.0,0.2,-0.1
2,0.3,0.3,0.3,0.0,0.1,0.3,0.2,0.4,0.3,0.1,...,-0.0,0.3,0.1,0.3,0.2,0.2,0.2,0.2,0.3,-0.3
3,0.1,0.2,0.0,-0.1,0.1,0.2,0.1,-0.0,-0.2,-0.1,...,0.0,0.2,-0.0,0.2,0.1,-0.0,-0.2,0.0,0.2,0.1
4,0.1,0.1,0.1,0.1,0.1,0.1,0.0,0.1,0.1,0.1,...,0.0,0.1,0.1,0.1,0.1,0.1,0.1,0.0,0.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-0.3,-0.2,-0.1,0.3,-0.0,0.2,0.0,0.0,-0.3,-0.3,...,-0.1,-0.1,-0.1,0.2,0.2,0.0,-0.3,-0.1,0.0,-0.1
196,0.1,0.1,-0.0,-0.1,-0.0,0.0,-0.1,-0.0,0.0,0.0,...,-0.1,-0.1,0.0,0.0,0.0,-0.0,-0.0,0.1,-0.0,-0.1
197,0.1,0.2,0.2,0.0,0.1,-0.4,-0.2,0.3,0.3,0.5,...,0.0,0.3,0.1,-0.0,0.1,0.4,0.1,-0.3,0.6,-0.1
198,0.3,0.2,0.2,0.2,-0.0,0.7,0.4,0.1,-0.0,-0.2,...,0.1,0.2,0.0,0.3,0.0,-0.0,0.2,0.3,-0.1,-0.1


Get the indexes of the top values, searches by lesson (column)

In [26]:
lesson_id = 0

user = max(results_df[(results_df[lesson_id] > .5)].index.values)
user

198

for lesson 0, user 198 is most likely to enjoy it

In [27]:
results_df.iloc[user,lesson_id] #checks it works

0.6

(it would be more helpful if it looks per user and returns lesson_ids)

In [38]:
for i in range(df_users.shape[0]):
    user_id = i
    lesson_id = results_df.idxmax(axis=1)[user_id]
    print(f'User {user_id} gets recommended lesson {lesson_id}')

User 0 gets recommended lesson 16
User 1 gets recommended lesson 1
User 2 gets recommended lesson 0
User 3 gets recommended lesson 3
User 4 gets recommended lesson 28
User 5 gets recommended lesson 0
User 6 gets recommended lesson 18
User 7 gets recommended lesson 1
User 8 gets recommended lesson 6
User 9 gets recommended lesson 16
User 10 gets recommended lesson 26
User 11 gets recommended lesson 26
User 12 gets recommended lesson 19
User 13 gets recommended lesson 2
User 14 gets recommended lesson 16
User 15 gets recommended lesson 26
User 16 gets recommended lesson 0
User 17 gets recommended lesson 18
User 18 gets recommended lesson 25
User 19 gets recommended lesson 2
User 20 gets recommended lesson 16
User 21 gets recommended lesson 27
User 22 gets recommended lesson 22
User 23 gets recommended lesson 26
User 24 gets recommended lesson 18
User 25 gets recommended lesson 8
User 26 gets recommended lesson 3
User 27 gets recommended lesson 19
User 28 gets recommended lesson 2
User 29

TODO:

sgd/funksvd for 'online' prediction

user lesson completed similarity with jacard similarity

```python
# calculate Jaccard similarity given two numpy array; 
##    and come up with a test case
## NOTE: use Jaccard when you have a binary (implicit) utility matrix!
##          E.g., Amazon’s purchase history.

def jaccard(a,b):
    return (a & b).sum() / (a | b).sum()

a = np.array([1, 1, 0, 0,1])
b = np.array([0, 1, 1, 0 ,1])
jaccard(a, b)
     

### Item-item similarity

cos sim to look at similarity across lessons

if a user looked at one look at the next five similar and recommend those

calculate similarity of one lesson, find items similar
add to total similarity matrix

In [40]:
lesson_tags = gen_lesson_tags()

In [41]:
lesson_tags.shape #30 lessons, 15 topics

(30, 15)

In [42]:
df_tags = pd.DataFrame(lesson_tags)

In [1]:
df_tags
df_tags['lesson_id'] = df_tags.index
df_tags #rows are lesson_id columns are latent topics found in all lessons, to various degrees

NameError: name 'df_tags' is not defined

In [50]:
sgd = FunkSGD(df_tags.values, 10 )

for _ in range(20):
    sgd.one_iter()
    
sgd.reconstruction_loss()

19.353017156321357

In [51]:
lesson_results_df = pd.DataFrame( (sgd.P @ sgd.Q.T).round(1) )
lesson_results_df.index.name = 'lesson_id'
#results_df = results_df.add_prefix('lesson_')
lesson_results_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
lesson_id,,,,,,,,,,,,,,,
0,0.0,-0.0,0.0,0.1,0.1,0.0,0.2,0.0,0.0,-0.1,-0.0,0.1,0.0,0.2,0.1
1,-0.1,0.2,0.0,0.1,-0.1,0.1,-0.0,0.1,-0.1,0.0,0.2,-0.1,-0.3,-0.2,-0.2
2,0.1,-0.1,0.3,0.1,0.1,-0.1,-0.2,-0.3,0.1,0.1,0.0,0.2,-0.0,0.1,0.5
3,0.1,-0.0,0.3,-0.0,0.0,0.1,0.1,0.1,0.1,-0.0,0.0,0.0,0.2,-0.1,0.2
4,0.0,0.1,0.1,0.2,0.1,0.0,-0.0,0.1,-0.0,-0.0,0.3,0.1,-0.1,0.1,0.2
5,0.0,-0.0,0.3,-0.0,0.1,-0.2,-0.2,-0.3,0.1,0.1,-0.0,0.2,-0.2,-0.1,0.3
6,0.0,0.2,0.0,-0.1,0.1,-0.1,-0.1,-0.2,0.0,0.3,-0.1,0.0,-0.2,0.0,-0.2
7,-0.0,0.0,0.1,0.3,0.1,0.0,0.1,-0.1,-0.1,-0.3,-0.1,0.2,0.1,0.5,0.3
8,-0.0,0.1,-0.2,0.1,0.1,0.1,0.1,-0.2,0.1,0.1,-0.1,0.0,-0.1,-0.1,-0.1


Using SVD

In [52]:
from sklearn.decomposition import NMF, TruncatedSVD

In [54]:
svd = TruncatedSVD(n_components=3)
svd.fit(df_tags)

# U and V are also common symbols; here I us P and Q to be consistent with http://nicolas-hug.com/blog/matrix_facto_3
P = svd.transform(df_tags)
Q = svd.components_.T

In [55]:
# RMSE with only three components is pretty small; we pretty much nailed it
((df_tags.values - P @ Q.T)**2).mean()**0.5

0.19789061001859287

In [56]:
P # lessons `u` vs. topic ????

array([[ 0.49737051,  0.12753733,  0.35496954],
       [ 0.1974889 ,  0.54885951, -0.39829229],
       [ 0.87357549, -0.47017386, -0.01342616],
       [ 0.28073007,  0.64729755, -0.31395724],
       [ 0.50264439, -0.10944646,  0.31839041],
       [ 0.69861851, -0.41115572, -0.2528622 ],
       [ 0.3681397 ,  0.5893847 , -0.54336872],
       [ 0.40751594,  0.26016712,  0.7138961 ],
       [ 0.4602764 ,  0.05584324, -0.04595493],
       [ 0.08258952,  0.06086003,  0.14212192],
       [ 0.65636312,  0.0912359 , -0.25630453],
       [ 0.31689161,  0.16602054,  0.11175531],
       [ 0.76894564, -0.46658584, -0.04196069],
       [ 0.57920626, -0.26527439,  0.11012718],
       [ 0.26332101,  0.42943558,  0.09956679],
       [ 0.270209  ,  0.12827192, -0.03750177],
       [ 0.31187232,  0.23199995, -0.32935324],
       [ 0.83237785, -0.39484143,  0.04348313],
       [ 0.83301145, -0.43350595, -0.13393201],
       [ 0.59195927,  0.15713041, -0.46072533],
       [ 0.3065447 ,  0.35238017,  0.697

In [57]:
Q # item `i` vs. topic ????

array([[ 0.25186618, -0.09835647, -0.09055366],
       [ 0.09724502,  0.40775906, -0.24951695],
       [ 0.16640692,  0.30496666, -0.06911917],
       [ 0.25274124,  0.24949235,  0.61283135],
       [ 0.20430278,  0.27587935, -0.27368964],
       [ 0.17469456,  0.26726032,  0.33619961],
       [ 0.11958406,  0.02709752, -0.03469463],
       [ 0.18197935,  0.00079869,  0.19970296],
       [ 0.24072161,  0.10254505, -0.21843079],
       [ 0.218725  ,  0.38526535, -0.35467215],
       [ 0.09716414,  0.07160004,  0.16720226],
       [ 0.30379353, -0.09163233,  0.14898078],
       [ 0.16232854,  0.21165773,  0.01181048],
       [ 0.16959939,  0.06328701,  0.26449031],
       [ 0.67322313, -0.54704925, -0.1625718 ]])

### Combine 2 recommenders for top recommendation

One possibility is use the item-item rec for cold start, ie users with less than say, 5 likes, and their behavior is unlikely to be indicative of their interests.

Another way to address cold start is have some type of intro sequence where user interests are inputed thru lesson choices early on, when they start a new language/learn a language basics.

Another way would be to input the top item similarity items in a mix with the top item recs. It all depends on the behavior of the users, if they use the thumbs up and down a lot then it can be a reliable predictor.